# Text classification with movie reivews
<hr>

이 notebook은 영화리뷰의 텍스트를 이용하여 긍정인지 부정인지를 분류한다. 이러한 긍, 부정 분류는 2중 분류 (binary, or two-class) 한 예시이고, 머신러닝 문제에서 넓게 적용할 수 있다.

사용될 데이터셋인 [IMDB dataset](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb)는:
- 50,000 영화 리뷰 포함하며
- training과 testing set들은 같은 각 set 내에서 같은 수의 긍정, 부정 리뷰를 담고있다.

여기서는 model을 구성하고 학습하기 하기 위한 high-level API인 `tf.keras`를 사용하며, `tf.keras`를 사용하여 text classification에 대해 더 심화된 가이드를 얻고 싶다면 [MLCC Text Classification Guide](https://developers.google.com/machine-learning/guides/text-classification/) 여기를 참고하라.

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

1.12.0


## Download the IMDB dataset
<hr>

IMDB dataset를 tensorflow로 다운받을 경우, 전처리가 되어있어 (단어들이 sequence 되어있음) 각 정수(각 정수는 dictionary에서 특정 단어를 표현)의 sequence로 변환되어있다.

다음 코드에서 IMDB dataset을 받아보자:

In [2]:
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

`num_words=1000`은 training set에서 자주 사용되는 단어 상위 10,000개를 추출하라는 argument이다. 이를 통해 데이터를 다루기 좋을 사이즈로 유지하기 위해서 드물게 사용되는 단어들은 제거됩니다.

## Explore the data
<hr>

data format을 유심히 보자. dataset이 전처리가 되어있기때문에, 각 예제는 영화 리뷰 단어들의 정수배열을 나타낸다. 각 label은 0 혹은 1인 정수를 가질 것이다. <br>*(0: negative review, 1: positive review)*

In [5]:
print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))

Training entries: 25000, labels: 25000


각 리뷰의 텍스트는 dictionary에서 특정 단어를 표현하는 정수로 변환되어있다. 첫번째 리뷰를 보면 알 수 있다:

In [6]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


각 영화리뷰들은 각각 다른 길이들을 가질것이고 아래의 코드로 첫번째와 두번째 리뷰를 보면 확인해볼 수 있다. 그런데 뉴럴넷의 input들은 고정된 길이를 가져야 하는데 이 문제는 일단 나중에 다룰 것이다.

In [8]:
len(train_data[0]), len(train_data[1])

(218, 189)

### convert the integers back to words

정수를 다시 텍스트로 변환해보자. dictionary object에 integer->string으로 매핑해주는 query를 날릴 수 있도록 helper 함수를 생성해보자:

In [23]:
# A dictionary mapping words to an interger index
word_index = imdb.get_word_index()

"""
print(word_index)
# { 'fawn': 34701, 'tsukino': 52006, 'nunnery': 52007, ...}
"""
# The first inddices are reserved
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2 # unknown
word_index["<UNUSED>"] = 3

# +코멘트) integer를 key로 두기 위해 key와 value를 뒤집는다.
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [30]:
reverse_word_index.get(333, '?') # 찾지 못하면 '?'로 둔다.

'second'

text를 넣었을때 각 단어들을 이어붙이고 그 사이에 띄어쓰기(' ')를 넣도록 함수를 짜면:

In [31]:
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

이제 `decode_review` 함수로 리뷰의 텍스트를 볼 수 있다.

In [32]:
decode_review(train_data[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

## Prepare the data
<hr>

각 리뷰(정수배열)는 뉴럴넷에 들어가기전, 반드시 tensor로 변환되어야 하고 다음의 방법들이 있다: 
- **array를 vector로변환**: one-hot encoding과 유사하게 있는 단어는 1을, 없는 단어는 0을 나타내는 vector로 변환 시킨다. 예를 들어, [3, 5]라는 한 리뷰 예제는 10,000차원(이전에 10,000의 단어만 추출하였으므로)의 vector가 되며, 이 vector에서 3과 5의 자리에만 1을, 나머지는 모두 0을 가진다. 그 후, floating point vector를 다룰 수 있는 Dense layer로 첫번째 layer를 생성하게 된다. 이런 방식은 `num_words * num_reivews` 크기의 matrix를 필요로 하는 메모리를 많이 필요로 하는 방식이다.
- **integer tensor**: 대안적으로, 같은 형태(길이)를 갖도록 array를 덧대고, `max_length * num_reviews` 크기의 integer tensor를 생성하는 방식이다. 이런 형태를 다룰 수 있는 embedding layer를 네트워크의 첫 layer에 두면 된다.

이 튜토리얼에서는 두번째 방식을 사용할 것이다.

리뷰들이 같은 길이를 가져야하므로, [pad_sequences](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences) 함수를 사용하여 길이를 규격화할 것이다.

In [34]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data, 
                                                       value=word_index["<PAD>"],
                                                       padding='post', 
                                                       maxlen=256)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, 
                                                       value=word_index["<PAD>"],
                                                       padding='post', 
                                                       maxlen=256)

다시 예제들을 보자:

In [37]:
len(train_data[0]), len(train_data[1])

(256, 256)

In [38]:
print(train_data[0])

[   1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941
    4  173   36  256    5   25  100   43  838  112   50  670    2    9
   35  480  284    5  150    4  172  112  167    2  336  385   39    4
  172 4536 1111   17  546   38   13  447    4  192   50   16    6  147
 2025   19   14   22    4 1920 4613  469    4   22   71   87   12   16
   43  530   38   76   15   13 1247    4   22   17  515   17   12   16
  626   18    2    5   62  386   12    8  316    8  106    5    4 2223
 5244   16  480   66 3785   33    4  130   12   16   38  619    5   25
  124   51   36  135   48   25 1415   33    6   22   12  215   28   77
   52    5   14  407   16   82    2    8    4  107  117 5952   15  256
    4    2    7 3766    5  723   36   71   43  530  476   26  400  317
   46    7    4    2 1029   13  104   88    4  381   15  297   98   32
 2071   56   26  141    6  194 7486   18    4  226   22   21  134  476
   26  480    5  144   30 5535   18   51   36   28  224   92   25  104
    4 

## Build the model
<hr>

뉴럴넷은 layer를 쌓아 형성되는데, 여기서 구조를 형성하는데 두가지의 주요한 결정사항이 있다:
- model에 layer를 얼마나 사용할 것인가?
- 각 layer에서 hidden unit은 얼마나 될 것인가?

이 예제에서는, input data는 word-indeces의 배열로 구성되며, 예측할 label은 0 혹은 1이다. 이 문제에서 model을 구축해보자:

In [40]:
# input shape is the vocabulary count used for the movie reviews(10,000 words)
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary() # +코멘트) 모델의 구조를 보여준다

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


1. **첫번째 레이어*: `Embedding layer`, 이 layer는 정수로 인코딩된 어휘들을 받고 각 단어색인에 해당하는 embedding vector를 찾는다. 이러한 vector들이 모델이 train됨에 때라 학습된다. vector들은 출력될 array에 차원을 추가하며, 그 결과 차원들은 `batch, sequence, embedding` 이다.
2. **두번째 레이어*: `GlobalAveragePooling1d`, 이 layer는 sequence dimension을 평균하여 각 예제에 고정적인 크기의 output vector를 리턴하는데, 이로 인해 model이 다양한 길이의 input을 다룰 수 있게 된다.
3. **세번째 레이어*: `Dense(16, activation=tf.nn.relu)`, 이전 레이어에서 받은 고정적인 output vector는 16개의 hidden unit으로 이루어진 fully-connected layer를 통과하게 된다.
4. **네번째 레이어*: `Dense(1, activation=tf.nn.sigmoid)`, 이전 레이어는 하나의 output node와 연결되고, `sigmoid` 활성함수(activation function)을 사용해 0과 1의 사이의 float을 얻게되며, 이 float는 확률 혹은 confidence level을 의미한다.

### Hidden units

### Loss function and optimizer

## Create a validation set
<hr>

## Train the model
<hr>

## Evaluate the model
<hr>

## Create a graph of accuracy and loss over time
<hr>